In [1]:
from keras.models import  load_model
from keras.utils import np_utils
import glob
import numpy as np
import os
import sed_eval
import dcase_util
import pickle
#%run shared_functions.ipynb

from PIL import Image

Using TensorFlow backend.


In [3]:

def getStackFeatMat(scene):
    with open(scene, "rb" ) as scenesample:
        l,fv = pickle.load(scenesample)
    return fv,l

def reduceLabels(labels):
    r,c=labels.shape 
    npOfLabels=np.zeros(c)
    for j in range(0,c):
        if np.sum(labels[:,j])>0:
            npOfLabels[j]=1
    return npOfLabels


def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        if 'train' in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return validationlist,testlist

labelRef={'barcelona':0, 'helsinki':1, 'london':2, 'paris':3, 'stockholm':4, 'vienna':5, 
          'airport':6, 'shopping_mall':7,'metro_station':8,'street_pedestrian':9,'public_square':10,
          'street_traffic':11,'tram':12,'bus':13,'metro':14,'park':15}

nb_classes=len(labelRef.keys())
def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[1]
        Y_[i,labelRef[location]]=1
        scene=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_[i,labelRef[scene]]=1
    return X_, Y_
validationlist,testlist=fileLists()
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)

validationfiles:  2518
testfiles:  2518
data obtained


In [6]:
#evaluate performance of each model
path='models\\multi_label.hdf5'
model=load_model(path)
nb_classes=16
#predictions=np.zeros((Y_test.shape))
#print(predictions.shape)
#print('**')
#predictions=model.predict(X_test, verbose=1, batch_size=8) #
#asc(predictions,'test')
#print("FIIIINESSSHED :-D")

evaluation=model.evaluate(X_test, Y_test, verbose=1,batch_size=16)
print(model.metrics_names)
print(evaluation)

2518/2518 [==============================] - ETA: 30 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 1ms/step
['loss', 'binary_accuracy']
[0.31405577326313289, 0.89706612386052054]


In [8]:

print(model.metrics_names)
print(evaluation)

['loss', 'binary_accuracy']
[0.31405577326313289, 0.89706612386052054]


In [12]:
predictions=model.predict(X_test, verbose=1, batch_size=16)

2518/2518 [==============================] - ETA: 31 - ETA: 10 - ETA: 7 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 1ms/step


In [33]:
print(Y_test.shape)

(2518, 16)


In [43]:
binarised = hv(predictions)
print("Both scene and city have to be correct, joint:")
asc(binarised,Y_test, joint=True)
print('location:')
asc(binarised[:,0:6],Y_test[:,0:6])
print('scene:')
asc(binarised[:,6:],Y_test[:,6:])

Both scene and city have to be correct, joint:
*** ASC:
correct scenes: 312
 Accuracy:
0.12390786338363781
location:
*** ASC:
correct scenes: 1076
 Accuracy:
0.4273232724384432
scene:
*** ASC:
correct scenes: 814
 Accuracy:
0.3232724384432089


0.3232724384432089

In [19]:
locationLabelRef={'barcelona':0, 'helsinki':1, 'london':2, 'paris':3, 'stockholm':4, 'vienna':5}
sceneLabelRef={'airport':0, 'shopping_mall':1,'metro_station':2,'street_pedestrian':3,'public_square':4,
          'street_traffic':5,'tram':6,'bus':7,'metro':8,'park':9}

nb_location_classes=len(locationLabelRef.keys())
nb_scene_classes=len(sceneLabelRef.keys())

def getTaskSeparateData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_s=np.zeros(((len(flist)),nb_scene_classes))
    Y_l=np.zeros(((len(flist)),nb_location_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[1]
        Y_l[i,locationLabelRef[location]]=1
        scene=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_s[i,sceneLabelRef[scene]]=1
    return X_,Y_s,Y_l
X_test,Y_test_s,Y_test_l=getTaskSeparateData(testlist)
X_test=np.expand_dims(X_test,axis=3)
scene_evaluation=model.evaluate(X_test,Y_test_s,verbose=1,batch_size=32)
location_evaluation=model.evaluate(X_test,Y_test_l,verbose=1,batch_size=32)
print(model.metrics_names)
print(scene_evaluation)
print(location_evaluation)

ValueError: Error when checking target: expected dense_4 to have shape (16,) but got array with shape (10,)

In [42]:
def hv(preds):
    scene=[]
    s,r=preds.shape
    hv=np.zeros((s,r))
    for i in range(0,s):
        for j in range(0,r):
             if preds[i,j]>globalthreshold:
                hv[i,j]=1
    return hv

#print(predictions.shape)
globalthreshold=0.4
#test_scenes=hv(predictions)
#print(test_scenes.shape)
#print(sum(test_scenes))

def asc(scene,Y_test,joint=False):
    correct=0  
    for idx,s in enumerate(scene):
        
        if joint:
            match=0
            for j,element in enumerate(s):
                if element==Y_test[idx][j] and element==1:  #element is the predicted state e.g. 1 for present, is it also 1 in gt
                    match=match+1
            if match==2: #has matched both scene and location
                correct=correct+1
        else:
            match=False
            for j,element in enumerate(s):
                if element==Y_test[idx][j] and element==1:  #element is the predicted state e.g. 1 for present, is it also 1 in gt
                    match=True
            if match: #has matched both scene and location
                correct=correct+1
    print('*** ASC:')
    print('correct scenes: ' +str(correct)+ '\n Accuracy:')
    print(correct/Y_test.shape[0]) 
    return correct/Y_test.shape[0]

#acc=asc(predictions)

In [ ]:
import itertools
def plot_confusion_matrix(mat, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          output = None):
    
    size = 3.5 + len(classes) * 0.5
    plt.figure(figsize=(size, size))
    plt.imshow(mat, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, horizontalalignment='right')
    plt.yticks(tick_marks, classes)

    fmt = '.0f' 
    thresh = mat.max() / 2.
    for i, j in itertools.product(range(mat.shape[0]), range(mat.shape[1])):
        plt.text(j, i, format(mat[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if mat[i, j] > thresh else "black")

    plt.tight_layout()
    #plt.ylabel('True label')
    #plt.xlabel('Predicted label')
    
    if output is not None:
        plt.savefig(output, bbox_inches='tight')
        
#strip out non-supermarket events

scene_mat=np.random.randint(9, size=(6, 6)) #need actual class values for this!
classes=[labelRef.keys()]
plot_confusion_matrix(scene_mat, classes, title='', output='scenes.pdf')
